In [36]:
# importing the libraries
import pandas as pd
import numpy as np
from tqdm import tqdm

# for reading and displaying images
from skimage.io import imread
from skimage.transform import resize
import matplotlib.pyplot as plt
# %matplotlib inline

# for creating validation set
from sklearn.model_selection import train_test_split

# for evaluating the model
from sklearn.metrics import accuracy_score

# PyTorch libraries and modules
import torch
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD

# torchvision for pre-trained models
from torchvision import models
import argparse
from collections import Counter
import time

import numpy as np

In [37]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [38]:
import os

import cv2
import pandas as pd

In [39]:
    label = 3
    df = pd.read_csv("/content/gdrive/MyDrive/mini-MIAS/data_description.csv", header=None, index_col=0)
    df[label].fillna('N', inplace=True)  # Empty values correspond to normal cases ('M'=malignant, 'B'=benign).
    df[3].str.strip()  # Strip leading and trailing spaces in label column.

    for img_pgm in os.listdir("/content/gdrive/MyDrive/mini-MIAS/images_original"):
        if img_pgm.endswith(".pgm"):
            img = cv2.imread("/content/gdrive/MyDrive/mini-MIAS/images_original/{}".format(img_pgm))
            img_name = img_pgm.split(".")[0]
            label = df.loc[img_name].loc[3]
            if label == 'N':
                new_path = "/content/gdrive/MyDrive/mini-MIAS/images_processed/normal_cases/{}.png".format(img_name)
            elif label == 'B ':
                new_path = "/content/gdrive/MyDrive/mini-MIAS/images_processed/benign_cases/{}.png".format(img_name)
            elif label == 'M ':
                new_path = "/content/gdrive/MyDrive/mini-MIAS/images_processed/malignant_cases/{}.png".format(img_name)
            cv2.imwrite(new_path, img)
            print("Converted {} from PGM to PNG ({} case).".format(img_pgm, label))
    print("Finished converting and sorting dataset.") 


# if __name__ == '__main__':
#     main()

Converted mdb008.pgm from PGM to PNG (N case).
Converted mdb009.pgm from PGM to PNG (N case).
Converted mdb006.pgm from PGM to PNG (N case).
Converted mdb001.pgm from PGM to PNG (B  case).
Converted mdb010.pgm from PGM to PNG (B  case).
Converted mdb004.pgm from PGM to PNG (N case).
Converted mdb012.pgm from PGM to PNG (B  case).
Converted mdb011.pgm from PGM to PNG (N case).
Converted mdb015.pgm from PGM to PNG (B  case).
Converted mdb005.pgm from PGM to PNG (B  case).
Converted mdb017.pgm from PGM to PNG (B  case).
Converted mdb013.pgm from PGM to PNG (B  case).
Converted mdb002.pgm from PGM to PNG (B  case).
Converted mdb007.pgm from PGM to PNG (N case).
Converted mdb018.pgm from PGM to PNG (N case).
Converted mdb014.pgm from PGM to PNG (N case).
Converted mdb003.pgm from PGM to PNG (N case).
Converted mdb019.pgm from PGM to PNG (B  case).
Converted mdb016.pgm from PGM to PNG (N case).
Converted mdb025.pgm from PGM to PNG (B  case).
Converted mdb023.pgm from PGM to PNG (M  case).
Co

In [40]:
print(os.listdir('/content/gdrive/MyDrive/mini-MIAS/images_processed'))

['benign_cases', 'normal_cases', 'malignant_cases']


In [1]:
from torchvision import transforms, datasets, models

image_transforms = {
    # Train uses data augmentation
    'train':
    transforms.Compose([
        #transforms.RandomResizedCrop(size=256, scale=(0.95, 1.0)),
        #transforms.RandomRotation(degrees=15),
        # transforms.ColorJitter(),
        #transforms.RandomHorizontalFlip(),
        #transforms.CenterCrop(size=224),  # Image net standards
        transforms.Resize((299,299)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])  # Imagenet standards
    ])}

In [43]:
def imshow_tensor(image, ax=None, title=None):

    if ax is None:
        fig, ax = plt.subplots()

    # Set the color channel as the third dimension
    image = image.numpy().transpose((1, 2, 0))

    # Reverse the preprocessing steps
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean

    # Clip the image pixel values
    image = np.clip(image, 0, 1)

    ax.imshow(image)
    plt.axis('off')

    return ax, image

In [1]:
'''
transform = image_transforms['train']
plt.figure(figsize=(24, 24))

for i in range(16):
    ax = plt.subplot(4, 4, i + 1)
    _ = imshow_tensor(transform(img), ax=ax)

plt.tight_layout()
'''

"\ntransform = image_transforms['train']\nplt.figure(figsize=(24, 24))\n\nfor i in range(16):\n    ax = plt.subplot(4, 4, i + 1)\n    _ = imshow_tensor(transform(img), ax=ax)\n\nplt.tight_layout()\n"

In [2]:
from torch.utils.data import DataLoader, sampler, random_split

batch_size = 1

all_data = datasets.ImageFolder(r'C:\Users\saikat\Dropbox\Deep Learning in Medical Images\mini-MIAS\images_processed',
                                transform=image_transforms['train'])
#train_data_len = int(len(all_data)*0.8)
#valid_data_len = int((len(all_data) - train_data_len)/2)
#test_data_len = int(len(all_data) - train_data_len - valid_data_len)
#train_data, val_data, test_data = random_split(all_data, [train_data_len, valid_data_len, test_data_len])
#train_data.dataset.transform = image_transforms['train']
#val_data.dataset.transform = image_transforms['val']
#test_data.dataset.transform = image_transforms['test']
#print(len(train_data), len(val_data), len(test_data))

data_loader = DataLoader(all_data, batch_size=batch_size, shuffle=True)
##val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=True)
#test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

In [11]:
trainiter = iter(data_loader)
features_train, labels_train = next(trainiter)
print(features_train.shape, labels_train.shape)

torch.Size([1, 3, 299, 299]) torch.Size([1])


In [16]:
model = models.vgg16_bn(pretrained=True)


In [17]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model=model.to(device)


In [18]:
model.features

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): ReLU(inplace=True)
  (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (9): ReLU(inplace=True)
  (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (12): ReLU(inplace=True)
  (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (14): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 

In [19]:
import numpy as np
features=np.zeros((len(data_loader),41473))

i=0
for inputs, labels in data_loader:
    inputs = inputs.to(device)
    outputs = model.features(inputs).detach().numpy()
    outputs=np.resize(outputs,(1,512*9*9))
    print(outputs.shape,type(outputs))
    #features[i,0:41472]=features[i,0:41472]+outputs
    #features[i,41472]+=labels.numpy()
    print(i)
    i=i+1
    
    
                 
                
                 


(1, 41472) <class 'numpy.ndarray'>
0
(1, 41472) <class 'numpy.ndarray'>
1
(1, 41472) <class 'numpy.ndarray'>
2


KeyboardInterrupt: 

In [15]:

import pandas as pd
df = pd.DataFrame (features)
filepath = 'MIAS_features_VGG_19_bn_generic.xlsx'
df.to_excel(filepath, index=False)

ValueError: This sheet is too large! Your sheet size is: 321, 41473 Max sheet size is: 1048576, 16384

In [53]:
# model.classifier[6] = Sequential(
#                       Linear(4096, 2))
# for param in model.classifier[6].parameters():
#     param.requires_grad = True